# 1D1. Práctica: Cálculo de Información Mutua

### Alumno: **Luis Fernando Izquierdo Berdugo**
### Materia: **Procesamiento de Información**
### Fecha: **10 de Septiembre de 2024**

**Instrucciones**:

Preprocesar el texto y convertirlo a minúsculas, quitar acentos y los siguientes caracteres:

`;:,.\\-\"'/()[]¿?¡!{}~<>|«»-—’\t\n\r`

Encontrar las asociaciones de palabras más significativas, usando la medida de información mutua para cada conjunto de datos. Cada conjunto de datos se deberá procesar por separado. 

Los archivos son:
- `archivo_emojis_Proceso.csv`
- `archivo_emojis_Elpais.csv`
- `archivo_emojis_Elfinanciero.csv`

Usar el contenido de la columna ‘title’. Se pueden leer con:

```python
data = pd.read_csv("<ruta>/archivo_emojis_Elfinanciero.csv")

text = data['title'].to_list()

text = " ".join(text)
```

Crear un notebook que calcule la información mutua de un texto. Usar una ventana de 2 palabras, para calcular las probabilidades conjuntas $p(x,y)$.

Incluir las asociaciones más importantes, respecto a la medida de información mutua para cada conjunto de datos proporcionado. Es decir, se deben de calcular todas las asociaciones entre el vocabulario único. Mostrar las diez asociaciones más importantes, de acuerdo al conjunto de datos proporcionado, indicar su índice de información mutua.

## Preprocesamiento de los textos

In [122]:
import unicodedata
import re
import pandas as pd

def pre(text):
    text = text.lower()
    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    text = re.sub(r"[\t\n]", " ", text)
    text = re.sub(r"[;:,.\-\"‘/”(\)\[\]¿?¡!\{\}~<>|«»-—\’\r\`\´]", "", text)
    return text

In [123]:
data = pd.read_csv("archivo_emojis_Elfinanciero.csv")
f1 = data['title'].to_list()
f1 = " ".join(f1)
f1 = pre(f1)

data = pd.read_csv("archivo_emojis_Elpais.csv")
f2 = data['title'].to_list()
f2 = " ".join(f2)
f2 = pre(f2)

data = pd.read_csv("archivo_emojis_Proceso.csv")
f3 = data['title'].to_list()
f3 = " ".join(f3)
f3 = pre(f3)

In [124]:
from nltk import ngrams
from collections import Counter
import numpy as np

def infoMutua(text):
    palabras = text.split()
    bigramas = list(ngrams(palabras, 2))
    conteoBigramas = Counter(bigramas)
    df = pd.DataFrame(conteoBigramas.items(), columns=['bigrama', 'conteo'])
    totalBigramas = sum(conteoBigramas.values())
    df['p_xy'] = df['conteo'] / totalBigramas
     # Calcular las probabilidades marginales de cada palabra
    conteoPalabras = Counter(palabras)
    palabrasTotal = sum(conteoPalabras.values())
    df['p_x'] = df['bigrama'].apply(lambda bigrama: conteoPalabras[bigrama[0]] / palabrasTotal)
    df['p_y'] = df['bigrama'].apply(lambda bigrama: conteoPalabras[bigrama[1]] / palabrasTotal)

    # Calcular la información mutua
    df['infoMutua'] = df.apply(lambda row: np.log2(row['p_xy'] / (row['p_x'] * row['p_y'])), axis=1)

    return df

In [125]:
df1 = infoMutua(f1)
df1.nlargest(10, 'infoMutua')

,bigrama,conteo,p_xy,p_x,p_y,infoMutua
174,"(gustaba, coleccionar)",1,0.000003,0.000003,0.000003,18.306039
175,"(coleccionar, ranas)",1,0.000003,0.000003,0.000003,18.306039
438,"(creando, conexiones)",1,0.000003,0.000003,0.000003,18.306039
472,"(tul, septuplicar)",1,0.000003,0.000003,0.000003,18.306039
1499,"(2001, politizacion)",1,0.000003,0.000003,0.000003,18.306039
1862,"(audrey, hale)",1,0.000003,0.000003,0.000003,18.306039
1863,"(hale, poseia)",1,0.000003,0.000003,0.000003,18.306039
1967,"(walter, divino)",1,0.000003,0.000003,0.000003,18.306039
1968,"(divino, gaitan)",1,0.000003,0.000003,0.000003,18.306039
3098,"(viet, nam)",1,0.000003,0.000003,0.000003,18.306039


In [126]:
print(len(f1.split()))
print(df1['conteo'].sum())

324090
324089


In [127]:
df2 = infoMutua(f2)
df2.nlargest(10, 'infoMutua')

,bigrama,conteo,p_xy,p_x,p_y,infoMutua
391,"(metieron, punete)",1,0.000002,0.000002,0.000002,18.749555
493,"(tadao, ando)",1,0.000002,0.000002,0.000002,18.749555
1026,"(hiba, abouk)",1,0.000002,0.000002,0.000002,18.749555
1036,"(achraf, hakimi)",1,0.000002,0.000002,0.000002,18.749555
1625,"(choteo, joda)",1,0.000002,0.000002,0.000002,18.749555
1626,"(joda, mamar)",1,0.000002,0.000002,0.000002,18.749555
1864,"(antiwar, pictures)",1,0.000002,0.000002,0.000002,18.749555
1886,"(adnan, syeds)",1,0.000002,0.000002,0.000002,18.749555
2342,"(reid, letrista)",1,0.000002,0.000002,0.000002,18.749555
2352,"(procol, harum)",1,0.000002,0.000002,0.000002,18.749555


In [128]:
print(len(f2.split()))
print(df2['conteo'].sum())

440735
440734


In [129]:
df3 = infoMutua(f3)
df3.nlargest(10, 'infoMutua')

,bigrama,conteo,p_xy,p_x,p_y,infoMutua
61,"(&quotenemigos, potenciales&quot)",1,0.000018,0.000018,0.000018,15.736243
94,"(alteza, serenisima&quot)",1,0.000018,0.000018,0.000018,15.736243
149,"(negocia, ataja)",1,0.000018,0.000018,0.000018,15.736243
270,"(fertilizante, asipona)",1,0.000018,0.000018,0.000018,15.736243
271,"(asipona, cristian)",1,0.000018,0.000018,0.000018,15.736243
272,"(cristian, magazo)",1,0.000018,0.000018,0.000018,15.736243
369,"(zonas, controladas)",1,0.000018,0.000018,0.000018,15.736243
541,"(terminal, indios)",1,0.000018,0.000018,0.000018,15.736243
542,"(indios, verdes)",1,0.000018,0.000018,0.000018,15.736243
579,"(alimentarias, &quothumo)",1,0.000018,0.000018,0.000018,15.736243


Se limpian los datos eliminando "&quot"

In [130]:
f3 = re.sub("&quot", "", f3)
df3 = infoMutua(f3)
df3.nlargest(10, 'infoMutua')

,bigrama,conteo,p_xy,p_x,p_y,infoMutua
94,"(alteza, serenisima)",1,0.000018,0.000018,0.000018,15.736243
147,"(negocia, ataja)",1,0.000018,0.000018,0.000018,15.736243
268,"(fertilizante, asipona)",1,0.000018,0.000018,0.000018,15.736243
269,"(asipona, cristian)",1,0.000018,0.000018,0.000018,15.736243
270,"(cristian, magazo)",1,0.000018,0.000018,0.000018,15.736243
367,"(zonas, controladas)",1,0.000018,0.000018,0.000018,15.736243
539,"(terminal, indios)",1,0.000018,0.000018,0.000018,15.736243
540,"(indios, verdes)",1,0.000018,0.000018,0.000018,15.736243
577,"(alimentarias, humo)",1,0.000018,0.000018,0.000018,15.736243
593,"(asesores, “planes)",1,0.000018,0.000018,0.000018,15.736243
